In [24]:
import scipy
from scipy.linalg import expm
import scipy.sparse as sparse
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis, spin_basis_1d # bases
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [25]:
L = 5 #system size
M = 0.1 #fermion mass
N=1 #number of layers
l=1 #spin length
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [26]:
#construct basis
basis=spin_basis_1d(L=L,S = str(l))
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
       0.         |2 2 2 2 2>           242  
       1.         |2 2 2 2 1>           241  
       2.         |2 2 2 2 0>           240  
       3.         |2 2 2 1 2>           239  
       4.         |2 2 2 1 1>           238  
       5.         |2 2 2 1 0>           237  
       6.         |2 2 2 0 2>           236  
       7.         |2 2 2 0 1>           235  
       8.         |2 2 2 0 0>           234  
       9.         |2 2 1 2 2>           233  
      10.         |2 2 1 2 1>           232  
      11.         |2 2 1 2 0>           231  
      12.         |2 2 1 1 2>           230  
      13.         |2 2 1 1 1>           229  
      14.         |2 2 1 1 0>           228  
      15.         |2 2 1 0 2>           227  
      16.         |2 2 1 0 1>           226  
      17.         |2 2 1 0 0>           225  
      18.         |2 2 0 2 2>           224  
      19.         |2 2 0 2 1>           223  
      20.  

In [27]:
#initialisation of the Gauss laws

liste = []
liste_m = []
gauss_law_map = []
Gauss_law = []

for i in range(L+1):
    liste.append([[1.,i]])
    liste_m.append([[-1.,i]])
    if i == 0:
        gauss_law_map.append([
            ["z",liste[0]]
        ])
    
    elif (i == L) & (i%2 == 0):
        gauss_law_map.append([
            ["z",liste_m[i-1]]
        ])
    
    elif (i == L) & (i%2 == 1):
        gauss_law_map.append([
            ["z",liste_m[i-1]],
            ["I",liste[i-1]]
        ])        
        
    elif i%2 == 0:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],
        ])
    elif i%2 == 1:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],  
            ["I",liste[i]]
        ])
    Gauss_law.append(hamiltonian(gauss_law_map[i],dynamic_list=[],basis=basis,**no_checks))


In [28]:
#basis vectors as arrays
basis_vectors = []
for index in range(basis.Ns):
    state = np.zeros(basis.Ns)
    state[basis.Ns-index-1] = 1.
    state = state.tolist()
    basis_vectors.append(state)

In [29]:
#projectors on g=1 for every site
Proj = [np.zeros((basis.Ns,basis.Ns)) for i in range(L+1)]
for i in range(L+1):
    for state in basis_vectors:
        state = np.array(state)
        if (Gauss_law[i].expt_value(state)>=1.-1e-8) & (Gauss_law[i].expt_value(state)<=1.+1e-8):
            Proj[i] += np.outer(state,state)

In [30]:
#initialisation of the unitarily transformed Hamiltonian

linear_term = []
quadratic_term = []
interaction_p = []
interaction_m = []


for i in range(L-1):
    linear_term.append([[0.5/np.sqrt(2),i]])
    quadratic_term.append([[(-1)**i/np.sqrt(2),i,i+1]])
    interaction_p.append([
        ["+z", quadratic_term[i]],
        ["+", linear_term[i]]
    ])
    interaction_m.append([
        ["-z", quadratic_term[i]],
        ["-", linear_term[i]]
    ])

linear_term.append([[0.5/np.sqrt(2),L-1]])
interaction_p.append([
    ["+", linear_term[L-1]]
])
interaction_m.append([
    ["-", linear_term[L-1]]
])

kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = [[2*(-1)**i*M,i] for i in range(L)]

kin_mass_map = [
            ["zz", kin_energy], 
            ["z", mass_term], 
]

H_int_p = []
H_int_m = []
interaction = [np.zeros((basis.Ns,basis.Ns)) for i in range(L)]
h_int = np.zeros((basis.Ns,basis.Ns))
for i in range(L):
    H_int_p.append(hamiltonian(interaction_p[i],dynamic_list=[],basis=basis,**no_checks))
    H_int_m.append(hamiltonian(interaction_m[i],dynamic_list=[],basis=basis,**no_checks))
    interaction[i] = (Proj[i]@H_int_p[i].toarray()@Proj[i+1]+Proj[i+1]@H_int_m[i].toarray()@Proj[i])

H_kin_mass = hamiltonian(kin_mass_map,dynamic_list=[],basis=basis,**no_checks)
h_kin_mass = H_kin_mass.toarray()

h_int = np.zeros((basis.Ns,basis.Ns),dtype = complex)

for i in range(L):
    h_int += interaction[i]


full_ham_matrix = h_kin_mass + h_int #Spin-1 Hamiltonian without the local contstaints


G=0
for i in range(L+1):
    G += 1e1*(Gauss_law[i]**2-Gauss_law[i])**2
constraint_full_matrix = G.toarray()

constrained_full_ham_matrix= full_ham_matrix + constraint_full_matrix #Spin-1 Hamiltonian with the local contstaints as a penalty term
print(full_ham_matrix)

[[2.7+0.j 0. +0.j 0. +0.j ... 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 2. +0.j 0. +0.j ... 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 2.3+0.j ... 0. +0.j 0. +0.j 0. +0.j]
 ...
 [0. +0.j 0. +0.j 0. +0.j ... 2.7+0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j ... 0. +0.j 2. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j ... 0. +0.j 0. +0.j 2.3+0.j]]


In [31]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian without the local contstaints
eigenval, eigenvec = np.linalg.eigh(full_ham_matrix)
print(eigenval[eigenval<=1e1])

[-1.0416023  -0.34551385 -0.34551385 -0.22065556 -0.13898669 -0.13898669
 -0.0142565   0.15174042  0.16101331  0.16101331  0.33967222  0.33967222
  0.33967222  0.33967222  0.33967222  0.35848584  0.53430948  0.53430948
  0.53479822  0.55681029  0.56101331  0.56101331  0.6         0.6
  0.6         0.63967222  0.63967222  0.69336559  0.69336559  0.72748942
  0.72748942  0.73967222  0.73967222  0.9         0.9         0.9
  0.9         0.9         0.9         0.9         0.9         0.93430948
  0.93430948  0.99336559  0.99336559  1.          1.          1.
  1.          1.          1.          1.03967222  1.03967222  1.03967222
  1.03967222  1.09336559  1.09336559  1.10404012  1.11592807  1.2
  1.2         1.2         1.2         1.2         1.2284373   1.276925
  1.276925    1.3         1.3         1.3         1.3         1.3
  1.3         1.3         1.3         1.3         1.3         1.3
  1.3         1.3         1.3         1.3         1.3         1.3
  1.39336559  1.39336559  1.43

In [32]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian with the local contstaints
eigenvalues, eigenvectors = np.linalg.eigh(constrained_full_ham_matrix)

In [33]:
#Gauge invariant part of the spectrum
print(eigenvalues[eigenvalues<=1e1])


[-1.0416023  -0.0142565   0.15174042  0.35848584  0.53479822  0.55681029
  1.10404012  1.11592807  1.2284373   1.48612285  1.75182104  1.76913858
  1.95519133  2.01107024  2.09579725  2.34850063  2.71262995  2.85799378
  3.6173529 ]


In [34]:
#initial state
string = ""
for i in range(L):
    string +="1"
psi_0 = np.zeros(basis.Ns)
i_0 = basis.index(string)
psi_0[i_0] = 1.

In [35]:
#cost function of the initial state
expt_value = 0
matvec_h = full_ham_matrix@psi_0
matvec_c = constraint_full_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*(matvec_h[i]+matvec_c[i])
print(np.real(expt_value))

0.0


In [36]:
expt_value = 0
matvec = full_ham_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*matvec[i]
print(expt_value)

0j


In [37]:
def rot_sigma(k,i,j,phi,theta):
    if k>=L:
        print("Warning! Rotations apply to k<=L-1!")
    sigma = np.zeros((3,3),dtype = complex)
    sigma[2-i][2-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2-j][2-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csc_matrix(sigma)
    rot_matrix = expm(-1j*theta*sigma/2)
    if k==0:
        tensprod = rot_matrix
        for i in range(L-1):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(3))
    else:
        tensprod = sparse.identity(3)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(3))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+1,L):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(3))
    return tensprod

def rot_ms(k,i,j,phi,theta):
    sigma = np.zeros((3,3),dtype = complex)
    sigma[2-i][2-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2-j][2-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csc_matrix(sigma)
    power_matrix = scipy.sparse.kron(sigma,sparse.identity(3))+scipy.sparse.kron(sparse.identity(3),sigma)
    rot_matrix = scipy.sparse.linalg.expm(-1j*power_matrix*theta/4)
    if k == 0:
        tensprod = rot_matrix
        for i in range(L-2):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(3))
    else:
        tensprod = sparse.identity(3)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(3))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+2,L):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(3))
    return tensprod

def rot_ms_e(i,j,phi,theta):
    ms_e = sparse.identity(basis.Ns,dtype = complex)
    for k in range(1,int(L/2)):
        ms_e = rot_ms(2*k,i,j,phi,theta)@ms_e
    return(ms_e)

def rot_ms_o(i,j,phi,theta):
    ms_o = sparse.identity(basis.Ns,dtype = complex)
    for k in range(int(L/2)-1):
        ms_o = rot_ms(2*k+1,i,j,phi,theta)@ms_o
    return(ms_o)

def rot_sigma_e(i,j,phi,theta):
    sigma_e = sparse.identity(basis.Ns,dtype = complex)
    for k in range(1,int(L/2)):
        sigma_e = rot_sigma(2*k,i,j,phi,theta)@sigma_e
    return(sigma_e)

def rot_sigma_o(i,j,phi,theta):
    sigma_o = sparse.identity(basis.Ns,dtype = complex)
    for k in range(int(L/2)):
        sigma_o = rot_sigma(2*k,i,j,phi,theta)@sigma_o
    return(sigma_o)

In [38]:
opt_params = []
function_values = []
def callback_function(x,fun,context):
    opt_params.append(x)
    function_values.append(fun)


In [39]:
full_ham_matrix = sparse.csr_matrix(full_ham_matrix)

In [40]:
#cost function definition
def cost_function_sigma(theta,psi,N):
    if len(theta)!=18*N:
        print("Warning: The number of variational parameters does not match the number of layers!")
    psi_var = psi
    for i in range(N):
        psi_var = rot_ms(0,0,2,0,theta[18*i+5])@rot_ms(0,1,2,0,theta[18*i+4])@rot_ms(0,0,1,0,theta[18*i+3])@rot_sigma(0,0,2,0,theta[18*i+2])@rot_sigma(0,1,2,0,theta[18*i+1])@rot_sigma(0,0,1,0,theta[18*i+0])@psi_var
        psi_var = rot_ms_o(0,2,0,theta[18*i+17])@rot_ms_o(1,2,0,theta[18*i+16])@rot_ms_o(0,1,0,theta[18*i+15])@rot_ms_e(0,2,0,theta[18*i+14])@rot_ms_e(1,2,0,theta[18*i+13])@rot_ms_e(0,1,0,theta[18*i+12])@rot_sigma_o(0,2,0,theta[18*i+11])@rot_sigma_o(1,2,0,theta[18*i+10])@rot_sigma_o(0,1,0,theta[18*i+9])@rot_sigma_e(0,2,0,theta[18*i+8])@rot_sigma_e(1,2,0,theta[12*i+7])@rot_sigma_e(0,1,0,theta[18*i+6])@psi_var
        psi_var = rot_ms(L-2,0,2,0,theta[18*i+5])@rot_ms(L-2,1,2,0,theta[18*i+4])@rot_ms(L-2,0,1,0,theta[18*i+3])@rot_sigma(L-1,0,2,0,theta[18*i+2])@rot_sigma(L-1,1,2,0,theta[18*i+1])@rot_sigma(L-1,0,1,0,theta[18*i+0])@psi_var
    expt_value = 0
    matvec = full_ham_matrix@psi_var
    for i in range(basis.Ns):
        expt_value += np.conj(psi_var[i])*matvec[i]
    cost = expt_value
    return np.real(cost)

In [41]:
#optimisation algorithm
from scipy.optimize import dual_annealing
import datetime
duan_ranges = []
for i in range(18*N):
    duan_ranges.append((-np.pi,np.pi))
resduan = 0
print(datetime.datetime.now())
resduan = dual_annealing(cost_function_sigma, duan_ranges, args = (psi_0,N))
print(datetime.datetime.now())

2022-03-07 11:06:53.069930
2022-03-07 12:13:41.573282


In [42]:
#results
print(resduan.fun)
print(resduan.x)


-0.8583492241835142
[-2.51762665  1.22917972 -2.53262179  2.42119712 -3.02578181 -3.14159265
  2.44121994 -2.7937789  -1.46787908  1.5367619   0.52338019  2.59272154
 -2.07245073  2.80396656  3.14159265 -2.81591884  3.14159265  3.14159265]


In [43]:
#fidelity of the result
theta = resduan.x

psi_var = psi_0
for i in range(N):
    psi_var = rot_ms(0,0,2,0,theta[18*i+5])@rot_ms(0,1,2,0,theta[18*i+4])@rot_ms(0,0,1,0,theta[18*i+3])@rot_sigma(0,0,2,0,theta[18*i+2])@rot_sigma(0,1,2,0,theta[18*i+1])@rot_sigma(0,0,1,0,theta[18*i+0])@psi_var
    psi_var = rot_ms_o(0,2,0,theta[18*i+17])@rot_ms_o(1,2,0,theta[18*i+16])@rot_ms_o(0,1,0,theta[18*i+15])@rot_ms_e(0,2,0,theta[18*i+14])@rot_ms_e(1,2,0,theta[18*i+13])@rot_ms_e(0,1,0,theta[18*i+12])@rot_sigma_o(0,2,0,theta[18*i+11])@rot_sigma_o(1,2,0,theta[18*i+10])@rot_sigma_o(0,1,0,theta[18*i+9])@rot_sigma_e(0,2,0,theta[18*i+8])@rot_sigma_e(1,2,0,theta[12*i+7])@rot_sigma_e(0,1,0,theta[18*i+6])@psi_var
    psi_var = rot_ms(L-2,0,2,0,theta[18*i+5])@rot_ms(L-2,1,2,0,theta[18*i+4])@rot_ms(L-2,0,1,0,theta[18*i+3])@rot_sigma(L-1,0,2,0,theta[18*i+2])@rot_sigma(L-1,1,2,0,theta[18*i+1])@rot_sigma(L-1,0,1,0,theta[18*i+0])@psi_var
print(np.abs(np.dot(np.conj(psi_var),eigenvectors[:,0]))**2)
expt_value = 0
matvec = full_ham_matrix@psi_var
for i in range(basis.Ns):
    expt_value += np.conj(psi_var[i])*matvec[i]
print(np.real(expt_value))

0.8634782236187499
-0.8583492241835142
